In [1]:
import pandas as pd

In [2]:
import apache_beam as beam
from apache_beam.io import jdbc ,ReadFromText,WriteToText


In [23]:
import psycopg2

In [29]:
from apache_beam.io.gcp.gcsfilesystem import GCSFileSystem
from google.cloud import storage


In [ ]:
pip install openpyxl

In [ ]:
pip install google-cloud-storage

In [22]:
pip install psycopg2-binary

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /jupyter/.kernels/apache-beam-2.46.0/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
items=[ 'id',
    'name',
    'domain',
    'year_of_experience',
    'education',
    'dob',
    'skill set',
    'expected_salary',
    'email',
    'mobilenumber',
    'country code',
    'availability date',
    'gender',
    'ready to relocate',
    'country',
    'created at',
    'updated at',
    'deleted at',
    'city/town',
    'state']

In [25]:
project_id = 'fine-program-384407'
bucket_name = 'myfiles-test'
file_pattern = 'gs://{}/my_dataframe*.csv'.format(bucket_name)
file_patterntxt = 'gs://{}/*.txt'.format(bucket_name)
connection_name = 'fine-program-384407:asia-south1:mydb'
databasex='mydb'
database_name = 'postgres'
username = 'postgres'
password = ""
table_name = 'target_table'


options = {
    'project': project_id,
    'region': 'us-central1',
    'runner': 'DataflowRunner',
    'job_name': 'load-csv-to-postgres',
    'temp_location': 'gs://{}/temp'.format(bucket_name),
    'staging_location': 'gs://{}/staging'.format(bucket_name),
    'requirements_file' : 'requirements.txt'
}

In [26]:
src_bucket_name = bucket_name
src_folder_name_fb = "source_facebook/"
src_folder_name_gf = "source_google/"
dest_bucket_name = bucket_name
dest_folder_name = "Loaded/"


In [30]:
client = storage.Client()
# Create a list to store the dataframes of all Excel files
all_dataframes = []

# Get a list of all the blobs (files) in the source folder of the source bucket
src_bucket = client.bucket(src_bucket_name)
src_folder = src_bucket.blob(src_folder_name_fb)
all_blobs = list(src_bucket.list_blobs(prefix=src_folder.name))


In [31]:
all_dataframes_gf = []

src_folder = src_bucket.blob(src_folder_name_gf)
all_blobs_gf = list(src_bucket.list_blobs(prefix=src_folder.name))

In [32]:


# Loop through each blob (file) in the list
for blob in all_blobs:
    # Check if the blob is an Excel file
    if blob.name.endswith(".xlsx"):
        # Load the Excel file into a Pandas dataframe
        excel_data = blob.download_as_string()
        df = pd.read_excel(excel_data, engine="openpyxl")
        all_dataframes.append(df)
        
        # Move the file to the destination folder of the destination bucket
        src_bucket.rename_blob(blob, dest_folder_name+all_blobs[1].name.split("/")[-1])
        #src_bucket.copy_blob(blob.name, dest_bucket_name, dest_folder_name + blob.name.split("/")[-1])
        #blob.delete()
all_blobs=[]

In [33]:


# Loop through each blob (file) in the list
for blob in all_blobs_gf:
    # Check if the blob is an Excel file
    if blob.name.endswith(".xlsx"):
        # Load the Excel file into a Pandas dataframe
        excel_data = blob.download_as_string()
        df = pd.read_excel(excel_data, engine="openpyxl")
        all_dataframes_gf.append(df)
        
        # Move the file to the destination folder of the destination bucket
        src_bucket.rename_blob(blob, dest_folder_name+all_blobs_gf[1].name.split("/")[-1])
        #src_bucket.copy_blob(blob.name, dest_bucket_name, dest_folder_name + blob.name.split("/")[-1])
        #blob.delete()

In [34]:
final_df_fb = pd.concat(all_dataframes, ignore_index=True)

,what_is_your_expected_salary?,from_where_you_heard_about_us?,years_of_experience_(in_field_of_sales_and_marketing),city/town,state,email,full_name,phone_number
0,"more_than_18,000",instagram,2 year,Jaipur,Rajasthan,maheshjoshi6350@gmail.com,Mahesh Narayan Joshi,916350354949
1,"more_than_18,000",facebook,1,Delhi,Delhi,princevikas759@gmail.com,Prince Vikas,919917884107
2,"more_than_18,000",instagram,Fresher,jaipur,Rajasthan,bannygoodboy1997@gmail.com,@Bhanu_Pratap,919653789451


In [60]:
data_fb

,id,name,domain,year_of_experience,education,dob,skill set,expected_salary,email,mobilenumber,country code,availability date,gender,ready to relocate,country,created at,updated at,deleted at,city/town,state
0,,Mahesh Narayan Joshi,,2 year,,,,"more_than_18,000",maheshjoshi6350@gmail.com,916350354949,,,,,,,,,Jaipur,Rajasthan
1,,Prince Vikas,,1,,,,"more_than_18,000",princevikas759@gmail.com,919917884107,,,,,,,,,Delhi,Delhi
2,,@Bhanu_Pratap,,Fresher,,,,"more_than_18,000",bannygoodboy1997@gmail.com,919653789451,,,,,,,,,jaipur,Rajasthan


In [35]:
final_df_gf = pd.concat(all_dataframes_gf, ignore_index=True)

In [59]:
final_df_gf

,id,Name,Contact Number (WhatsApp),Education,Years Of Experience (in field of Sales and Marketing),City/Town,What is your expected salary?,State,From where you heard about us?
0,1,Aditya Goyal,7248341035,Graduation,3+,Vikasnagar Dehradun,"More than 18,000",Uttarakhand,NaN
1,2,Devashish Chhetri,8077246893,Completed B.Sc-IT,1,Dehradun,"More than 18,000",Uttarakhand,NaN
2,3,Deepak Singh dagur,6377762132,12 th pass,1,Nagar bharatpur,"More than 18,000",Rajasthan,NaN


In [36]:
data_fb= final_df_fb
data_google=final_df_gf

In [37]:
# Map the columns to the desired names
data_fb = data_fb.rename(columns={
    'what_is_your_expected_salary?': 'expected_salary',
    'years_of_experience_(in_field_of_sales_and_marketing)': 'year_of_experience',
    'full_name': 'name',
    'phone_number': 'mobilenumber'
})
# Map the columns to the desired names
data_google = data_google.rename(columns={

    'What is your expected salary?': 'expected_salary',
    'Years Of Experience (in field of Sales and Marketing)': 'year_of_experience',
    'Name': 'name',
    'Contact Number (WhatsApp)': 'mobilenumber',
    'Education':'education',
    'State' : 'state'
    
})

In [38]:
for i in items:
    if i not in data_google:
        data_google[i]=''
    if i not in data_fb:
        data_fb[i]=''


In [39]:
data_google['id']=''

In [44]:
data_google = data_google[[
    'id',
    'name',
    'domain',
    'year_of_experience',
    'education',
    'dob',
    'skill set',
    'expected_salary',
    'email',
    'mobilenumber',
    'country code',
    'availability date',
    'gender',
    'ready to relocate',
    'country',
    'created at',
    'updated at',
    'deleted at',
    'city/town',
    'state'
]]
'''id,name,domain,year_of_experience,education,dob,skill set,expected salary,email,mobile number,
country code,availability date,gender,ready to relocate,country,created at ,updated at,deleted at,city,state
'''
# Reorder the columns
data_fb = data_fb[[
    'id',
    'name',
    'domain',
    'year_of_experience',
    'education',
    'dob',
    'skill set',
    'expected_salary',
    'email',
    'mobilenumber',
    'country code',
    'availability date',
    'gender',
    'ready to relocate',
    'country',
    'created at',
    'updated at',
    'deleted at',
    'city/town',
    'state'
]]


In [45]:
df_union = pd.concat([data_fb, data_google])

In [46]:
df_union.reset_index(drop=True, inplace=True)
df_union =df_union.drop(columns=['id'])

In [47]:
df_union.replace(r'^\s*$', 'NULL', regex=True, inplace=True)
df_union.replace(r',', '', regex=True, inplace=True)

In [48]:
df_union

,name,domain,year_of_experience,education,dob,skill set,expected_salary,email,mobilenumber,country code,availability date,gender,ready to relocate,country,created at,updated at,deleted at,city/town,state
0,Mahesh Narayan Joshi,NULL,2 year,NULL,NULL,NULL,more_than_18000,maheshjoshi6350@gmail.com,916350354949,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Jaipur,Rajasthan
1,Prince Vikas,NULL,1,NULL,NULL,NULL,more_than_18000,princevikas759@gmail.com,919917884107,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Delhi,Delhi
2,@Bhanu_Pratap,NULL,Fresher,NULL,NULL,NULL,more_than_18000,bannygoodboy1997@gmail.com,919653789451,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,jaipur,Rajasthan
3,Aditya Goyal,NULL,3+,Graduation,NULL,NULL,More than 18000,NULL,7248341035,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Uttarakhand
4,Devashish Chhetri,NULL,1,Completed B.Sc-IT,NULL,NULL,More than 18000,NULL,8077246893,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Uttarakhand
5,Deepak Singh dagur,NULL,1,12 th pass,NULL,NULL,More than 18000,NULL,6377762132,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Rajasthan


In [49]:
import datetime

time_now  = datetime.datetime.now().strftime('%m_%d_%Y_%H_%M_%S') 

print(time_now)

04_30_2023_12_17_37


In [50]:
file_path = 'my_dataframe'+time_now+'.csv'
# Save the DataFrame to a CSV file
df_union.to_csv(file_path, index=False,header=False)

# Upload the CSV file to GCS
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.upload_from_filename(file_path)

In [62]:
load1 = ['Mahesh hi', 'NULL', '2 year', 'NULL', 'NULL', 'NULL', 'more_than_18000', 'maheshjoshi6350@gmail.com', '6350354949', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'Jaipur', 'Rajasthan']

In [63]:
load1[8]

'6350354949'

In [53]:

query1 = "WHERE mobile_number = {} ".format(load1[8])

In [54]:
query

NameError: name 'query' is not defined

In [64]:
load_to_postgres(load1, connection_name, database_name, username, password, table_name2)

UPDATE target_table SET name= %s,domain= %s,year_of_experience= %s,education= %s,        dob= %s,skill_set= %s,expected_salary= %s,email= %s,mobile_number= %s,        country_code= %s,availability_date= %s,gender= %s,ready_to_relocate= %s,        country= %s,created_at= %s,updated_at= %s,deleted_at= %s,city= %s,state= %s WHERE mobile_number = '6350354949' 


In [61]:
def load_to_postgres(batch, connection_name, database_name, username, password, table_name):
    import psycopg2
    import logging
    connection = psycopg2.connect(
        host='172.24.32.3',
        port= 5432,
        database=database_name,
        user=username,
        password=password
    )

    cursor = connection.cursor()
    cursor.execute(f"SELECT * FROM {table_name} WHERE mobile_number = %s", (batch[8],))
    existing_row = cursor.fetchone()
    if existing_row:
        query = "UPDATE {0} SET name= %s,domain= %s,year_of_experience= %s,education= %s,\
        dob= %s,skill_set= %s,expected_salary= %s,email= %s,mobile_number= %s,\
        country_code= %s,availability_date= %s,gender= %s,ready_to_relocate= %s,\
        country= %s,created_at= %s,updated_at= %s,deleted_at= %s,city= %s,state= %s WHERE mobile_number = '{1}' ".format(table_name,str(batch[8]))
        cursor.execute(query, list(batch))
        print(query)
        '''cursor.execute(f"UPDATE {table_name} SET name= %s,domain= %s,year_of_experience= %s,education= %s,\
        dob= %s,skill_set= %s,expected_salary= %s,email= %s,mobile_number= %s,\
        country_code= %s,availability_date= %s,gender= %s,ready_to_relocate= %s,\
        country= %s,created_at= %s,updated_at= %s,deleted_at= %s,city= %s,state= %s WHERE mobile_number = %s", list(batch))'''
    else:
        query = "INSERT INTO {} (name,domain,year_of_experience,education,dob,skill_set,expected_salary,email,mobile_number,\
        country_code,availability_date,gender,ready_to_relocate,country,created_at,updated_at,deleted_at,city,state) \
        VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s,%s,%s,%s)".format(table_name)
        logging.info('%s', query)
        logging.info('%s', list(batch))
        cursor.execute(query, list(batch))
        #if cu
    #for line in batch:
        # Split the line into fields
        #fields = line.split(',')

        # Check if the line already exists in the table
        #query = "SELECT COUNT(*) FROM {} WHERE id = %s".format(table_name)
        #cursor.execute(query, (batch[0],))

        #if cursor.fetchone()[0] == 0:
            # Insert the line into the table



    connection.commit()
    cursor.close()
    connection.close()


In [ ]:
'''
class MoveFile(beam.DoFn):
    def __init__(self, source_bucket, source_prefix, dest_bucket, dest_prefix):
        self.source_bucket = source_bucket
        self.source_prefix = source_prefix
        self.dest_bucket = dest_bucket
        self.dest_prefix = dest_prefix
        self.gcs = GCSFileSystem()

    def process(self, element):
        source_file = element.strip()
        source_path = f"gs://{self.source_bucket}/{self.source_prefix}/{source_file}"
        dest_path = f"gs://{self.dest_bucket}/{self.dest_prefix}/{source_file}"
        blob = storage.blob.Blob.from_string(source_path, self.gcs)
        self.gcs.rename(blob, dest_path)
        yield source_file'''

In [ ]:
'''def load_to_postgres(batch, connection_name, database_name, username, password, table_name):
    import psycopg2
    import logging
    connection = psycopg2.connect(
        host='172.24.32.3',
        port= 5432,
        database=database_name,
        user=username,
        password=password
    )

    cursor = connection.cursor()

    #for line in batch:
        # Split the line into fields
        #fields = line.split(',')

        # Check if the line already exists in the table
        #query = "SELECT COUNT(*) FROM {} WHERE id = %s".format(table_name)
        #cursor.execute(query, (batch[0],))
    cursor.execute(f"SELECT * FROM {table_name} WHERE id = %s", (batch[0],))
    existing_row = cursor.fetchone()
    if existing_row:
        cursor.execute(f"UPDATE {table_name} SET name = %s, age = %s WHERE id = %s", (row[1], row[2], row[0]))
    else:
        cursor.execute(f"INSERT INTO {table_name} (id, name, age) VALUES (%s, %s, %s)", row)
    
        #if cursor.fetchone()[0] == 0:
            # Insert the line into the table
    query = "INSERT INTO {} (app_name,click_event,color,country,customer_id,date,device_name,device_type,discount,email,\
    ip_address,ip_browser,is_ordered_flag,item_category,item_name,payment_method,\
    router_mac_address) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s,%s)".format(table_name)
    logging.info('%s', query)
    cursor.execute(query, list(batch))

    connection.commit()
    cursor.close()
    connection.close()
'''

In [65]:

with beam.Pipeline(options=beam.pipeline.PipelineOptions.from_dictionary(options)) as p:
    # Read CSV file from Cloud Storage
    data = p | ReadFromText(file_pattern) 

    # Transform data as necessary
    transformed_data = data | beam.Map(lambda row: row.split(','))

    # Write data to batch files in Cloud Storage
    '''batch_files = transformed_data | WriteToBatchFiles(
        file_path_prefix='gs://{}/output'.format(bucket_name),
        shard_name_template='',
        batch_size=1000
    )'''
    # Write transformed data to text file
    text_files = transformed_data | WriteToText(
        file_path_prefix='gs://{}/output'.format(bucket_name),
        file_name_suffix='.txt'
    )
    #datatxt = ReadFromText(file_patterntxt)

    # Read data from text file and load into PostgreSQL table
    _ = text_files |'LoadToPostgres' >> beam.ParDo(load_to_postgres, connection_name, database_name, username, password, table_name)
    # Move CSV file to archive folder
    #_ = data | beam.ParDo(MoveFile('myfiles-test', 'rut.csv', 'myfiles-test', 'archive'))


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /jupyter/.kernels/apache-beam-2.46.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /jupyter/.kernels/apache-beam-2.46.0/bin/python -m pip install --upgrade pip


In [ ]:
python -m plsqlload (4).ipynb\
    --runner DataflowRunner \
    --project fine-program-384407 \
    --staging_location gs://myfiles-test/staging \
    --temp_location gs://myfiles-test/temp \
    --template_location gs://myfiles-test/templates/TEMPLATE_NAME \
    --region REGION